In [5]:
import torch
from torch.nn import functional as F
import numpy as np

## [FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness](https://arxiv.org/abs/2205.14135)

### Forward Pass

In [271]:
'''
    Q: (batch, seq_len, dim)
    K: (batch, seq_len, dim)
    V: (batch, seq_len, dim)
    tau: softmax scaling
    is_causal: whether to use causal mask
    dropout_p: dropout probability
    
    return: (batch, seq_len, dim)
'''

def flash_attention(Q, K, V, M = 100_000, tau = 1, is_causal = False, dropout_p = 0, seed = 1234):
    b, N, d = Q.shape
    Bc = M // (4 * d)
    Br = min(Bc,d)
    O = torch.zeros_like(Q)
    l = torch.zeros(b, N)
    m = torch.ones(b, N) * -float('inf')
    Tr = N // Br + (1 if N % Br != 0 else 0)
    Tc = N // Bc + (1 if N % Bc != 0 else 0)

    for b_i in range(b):
        for j in range(Tc):
            N_c = Bc * j
            K_j = K[b_i,N_c:N_c + Bc]
            V_j = V[b_i,N_c:N_c + Bc]
            
            for i in range(Tr):
                N_r = Br * i
                Q_i = Q[b_i, N_r:N_r + Br]
                O_i = O[b_i, N_r:N_r + Br]
                l_i = l[b_i, N_r:N_r + Br]
                m_i = m[b_i, N_r:N_r + Br]

                S_ij = torch.matmul(Q_i, K_j.T) * tau
                
                if is_causal:
                    # Apply causal mask
                    r_max = N_c + Bc - N_r - 1
                    c_start = Bc - r_max
                    for r in range(min(r_max, S_ij.shape[0])):
                        S_ij[r, max(c_start + r, 0):] = torch.tensor(-float('inf'))

                m_ij = torch.max(S_ij, dim=1).values
                S_ij = S_ij - m_ij.unsqueeze(1)
                # Need to make sure that the exp is not nan, value of nan could be caused by applying causal mask
                S_ij[S_ij.isnan()] = 0
                P_ij = torch.exp(S_ij)
                # P_ij = torch.exp(torch.where(S_ij - m_ij.unsqueeze(1) != float('nan'), ))
                l_ij = torch.sum(P_ij, dim=1)                
                m_i_new = torch.max(m_i, m_ij)
                l_i_new = torch.exp(m_i - m_i_new) * l_i + torch.exp(m_ij - m_i_new) * l_ij
                if dropout_p > 0:
                    torch.manual_seed(seed)
                    P_ij = F.dropout(P_ij, p = dropout_p, training = True)
                
                O_i = torch.diag(1 / l_i_new) @ (torch.diag(l_i * torch.exp(m_i - m_i_new)) @ O_i
                                                 + torch.diag(torch.exp(m_ij - m_i_new)) @ torch.matmul(P_ij, V_j))

                O[b_i, N_r:N_r + Br] = O_i
                l[b_i, N_r:N_r + Br] = l_i_new
                m[b_i, N_r:N_r + Br] = m_i_new
               
    return O

def attention(Q, K, V, tau = 1, dropout_p = 0, is_causal = False):
    S = (Q @ K.transpose(1,2)) * tau
    if is_causal:
        S = torch.where(torch.tril(torch.ones_like(S)) != 0, S, torch.tensor(-float('inf')))  # apply causal mask
    P = torch.softmax(S, dim=-1)
    if dropout_p > 0:
        P = F.dropout(P, p = dropout_p, training = True)

    O = P @ V

    return O, P

#### Test

In [272]:
%%time

b, n, d = 16, 1024, 512
# b, n, d = 8, 128, 32

Q = torch.randn(b, n, d)
K = torch.randn(b, n, d)
V = torch.randn(b, n, d)

O, P = attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True)
O_flash = flash_attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True)


print(O[0][0][:10])
print(O_flash[0][0][:10])
print(f'Total difference: {torch.sum(torch.abs(O_flash - O))}')
print(O_flash.allclose(O, atol=1e-4))

O_flash = flash_attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True, dropout_p = 0.1)
print(O_flash[0][0][:10])

tensor([ 0.8029, -1.2119, -0.6681,  0.9253,  1.8810,  2.6240,  1.5704, -0.3023,
        -0.4101,  0.7639])
tensor([ 0.8029, -1.2119, -0.6681,  0.9253,  1.8810,  2.6240,  1.5704, -0.3023,
        -0.4101,  0.7639])
Total difference: 0.3089710474014282
True
tensor([ 0.8921, -1.3466, -0.7424,  1.0281,  2.0900,  2.9155,  1.7449, -0.3358,
        -0.4557,  0.8488])
CPU times: user 1min 9s, sys: 810 ms, total: 1min 10s
Wall time: 7.26 s


### Backward Pass

In [251]:
def flash_attention_grad(Q, K, V, O, dO, M = 100_000, tau = 1, is_causal = False, dropout_p = 0, seed = 1234):
    pass

def attention_grad(Q, K, V, P, dO):
    dV = P.transpose(1,2) @ dO
    dP = dO @ V.transpose(1,2)
    dS = P * (dP - (P * dP).sum(dim=-1).unsqueeze(-1))
    # dS = torch.zeros_like(dP)
    # for i in range(dP.shape[1]):
    #     for j in range(dP.shape[2]):
    #         dS[:, i, j] = P[:, i, j] * (dP[:, i, j] - (P[:,i] * dP[:,i]).sum(dim=1))       
    dQ = dS @ K
    dK = dS.transpose(1,2) @ Q

    return dQ, dK, dV

#### Test

In [253]:
b, n, d = 16, 1024, 512
# b, n, d = 8, 16, 4

Q = torch.randn(b, n, d)
K = torch.randn(b, n, d)
V = torch.randn(b, n, d)

O, P = attention(Q, K, V, tau=1/np.sqrt(d), is_causal = True)
dO = O * 0.1

dQ, dK, dV = attention_grad(Q, K, V, P, dO)
print(Q[0][0][:10], dQ[0][0][:10])
print(K[0][0][:10], dK[0][0][:10])
print(V[0][0][:10], dV[0][0][:10])

tensor([-0.1081, -0.0515,  0.2460,  0.5301,  0.0485,  0.9343,  0.6369, -0.2612,
        -0.9486,  0.8193]) tensor([-1.9719,  3.5232,  7.1646,  0.0403,  3.8813, -0.0787,  5.0678,  9.2380,
         3.3660, -1.1149])
tensor([ 1.2582, -0.9250, -0.5326, -1.2587,  0.9578, -0.0726, -0.6635, -1.1729,
        -0.8874, -0.4861]) tensor([ 4.2310, -3.4429,  3.2846, -1.6327, -0.9032, -6.1530, -7.0747,  3.0991,
        -3.9514,  1.9892])
tensor([ 0.6357,  0.3955, -2.4005,  0.0158,  0.4290, -0.0535, -0.2182,  0.7395,
         1.1625, -0.0243]) tensor([ 0.2419, -0.1313, -0.3112,  0.0349, -0.0551, -0.1749, -0.0601,  0.0838,
        -0.0333,  0.0851])


In [296]:
import torch
import torch.nn.functional as F


# Define a tensor
tensor = torch.randn(3, 3)

# Set the seed
torch.manual_seed(123)
# Apply dropout
print(F.dropout(tensor, p=0.5))

torch.manual_seed(123)
# Apply dropout
print(F.dropout(tensor, p=0.5))

tensor([[ 1.6042,  1.7630,  3.3717],
        [-1.5306,  0.8845, -2.1709],
        [-0.0000, -0.6152, -0.0000]])
tensor([[ 1.6042,  1.7630,  3.3717],
        [-1.5306,  0.8845, -2.1709],
        [-0.0000, -0.6152, -0.0000]])
